# Dokumentacja modułu do scrapowania plików PDF z witryn banków
#### Wprowadzenie
Kod ten jest przeznaczony do automatycznego pobierania plików PDF z witryn banków na podstawie określonych słów kluczowych i zabezpieczeń przed niechcianymi plikami. Wykorzystuje bibliotekę requests do pobierania zawartości stron internetowych, BeautifulSoup do analizy HTML, wget do pobierania plików, a także moduł pandas do manipulacji danymi w formie tabelarycznej.

Zaimplementowany moduł umożliwił znaczne ograniczenie pobieranych plików .pdf do tylko i wyłącznie użytecznych w procesie przetwarzania informacji nt. ofert depozytowych banków. Dobrym przykładem niech będzie strona banku Pekao S.A., na której znajduje się ponad 10000 plików .pdf możliwych do pobrania (w wyniku użycia modułu ograniczono tę liczbę do 3(!)).

#### Funkcje i kluczowe zmienne:
#### __extract_domain_name_from_url(url)
Ta funkcja ekstrahuje nazwę domeny z podanego adresu URL. Umożliwia to utworzenie unikalnej nazwy pliku PDF na podstawie domeny.

#### download_pdfs_from_url(keywords, stopwords, url, download_path='pdfs')
Główna funkcja kodu, która pobiera pliki PDF z podanej witryny bankowej na podstawie słów kluczowych. Wszystkie pobrane pliki są zapisywane w określonym katalogu.

- keywords: Lista słów kluczowych, których obecność jest wymagana w adresie URL pliku PDF.
- stopwords: Lista słów, których obecność w adresie URL pliku PDF powoduje jego pominięcie.
- url: Adres URL witryny bankowej.
- download_path: Ścieżka do katalogu, w którym mają być zapisywane pobrane pliki PDF.
#### Przykład użycia:

In [ ]:
csv_filepath = "./data/bank_adresses.csv"
df = pd.read_csv(filepath_or_buffer=csv_filepath, delimiter=';')

keywords = ["lokat", "oszcz", "depoz", "oproc"]
stopwords = ["kred", "reg", "kart", "oswiadcz", "inwalid", "toip", "oplat", "prowiz", "umow", "wnios",
             "wycof", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]

for label, row in df.iterrows():
    try:
        if row["url"]:
            download_pdfs_from_url(keywords, stopwords, str(row['url']), download_path='pdfs')
    except Exception as e:
        print(e)

#### Wymagania:
- requests: Do pobierania zawartości stron internetowych.
- BeautifulSoup: Do analizy HTML i wyszukiwania linków do plików PDF.
- wget: Do pobierania plików.
- pandas: Do manipulacji danymi w formie tabelarycznej.
#### Uwagi:
- Upewnij się, że katalog docelowy (pdfs) istnieje przed uruchomieniem kodu, w przeciwnym razie zostanie utworzony.
- W przypadku problemów związanych z dostępem do witryn internetowych lub nieprawidłowymi danymi w pliku CSV, kod może zgłaszać wyjątki, które warto monitorować w celu zrozumienia przyczyn.
- Staraj się przestrzegać zasad etycznych podczas automatycznego pobierania plików z witryn internetowych.

In [40]:
import os
import requests
from bs4 import BeautifulSoup
import wget
from urllib.parse import urljoin
from urllib.parse import urlparse
import pandas as pd

def __extract_domain_name_from_url(url):
    start_index = url.find("www.") + len("www.")
    end_index = url.find(".pl", start_index)
    
    if start_index != -1 and end_index != -1:
        return url[start_index:end_index]
    else:
        return None

def download_pdfs_from_url(keywords, stopwords, url, download_path='pdfs'):
    # Tworzymy katalog, jeśli nie istnieje
    if not os.path.exists(download_path):
        os.makedirs(download_path)

    # Pobieramy zawartość strony
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Szukamy linków do plików PDF
    pdf_links = soup.find_all('a', href=lambda href: href and href.lower().endswith('.pdf') and any(keyword.lower() in href.lower() for keyword in keywords) and not any(stopword.lower() in href.lower() for stopword in stopwords))
    files_counter = 1
    # Pobieramy pliki PDF
    for pdf_link in pdf_links:
        absolute_url = urljoin(url, pdf_link['href'])
        print(absolute_url)
        pdf_filename = "pdfs/" + __extract_domain_name_from_url(url) + "_" + str(files_counter) + ".pdf"
        print(f"Pobieranie: {pdf_filename}")
        files_counter = files_counter + 1
        wget.download(absolute_url, pdf_filename)

csv_filepath = "./data/bank_adresses.csv"
df : pd.DataFrame = pd.read_csv(filepath_or_buffer=csv_filepath, delimiter=';')

keywords = ["lokat", "oszcz", "depoz", "oproc"]
stopwords = ["kred", "reg", "kart", "oswiadcz", "inwalid", "toip", "oplat", "prowiz", "umow", "wnios",
             "wycof", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022"]

for label, row in df.iterrows():
    try:
        if row["url"]:
            download_pdfs_from_url(keywords, stopwords, str(row['url']), download_path='pdfs')
    except Exception as e:
        print(e)